In [1]:
%pip install langchain_community beautifulsoup4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [beautifulsoup4]
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import WebBaseLoader
from bs4 import SoupStrainer


url = "https://m.sports.naver.com/wfootball/article/450/0000138646"

parse_filter = SoupStrainer("div", class_="ArticleContent_comp_article_content__luOFM")

loader = WebBaseLoader(
    web_path=url,
    bs_kwargs={"parse_only" : parse_filter}
)

documents = loader.load()

if documents : 
    first_page = documents[0]
    print("=======첫번째 페이지 상세 정보=======")
    print(f"Source : {first_page.metadata['source']}")
    print(first_page.page_content[:300])

=======첫번째 페이지 상세 정보=======
Source : https://m.sports.naver.com/wfootball/article/450/0000138646
인터 마이애미 리오넬 메시, LAFC 손흥민. /사진=게티이미지코리아[STN뉴스] 반진혁 기자 | 손흥민의 영입 효과를 제대로 누리고 있는 LAFC다.손흥민은 토트넘 홋스퍼 10년 생활을 정리한 후 새로운 도전에 나섰고 LAFC와 동행을 약속했다.LAFC는 지난 7일 손흥민 영입을 발표했다. 계약 기간은 2027년까지이며, 2028, 2029년 6월까지 추가 옵션이 있다.LAFC는 지난 10일 미국 시카고에 위치한 시트긱 스타디움에서 치러진 시카고 파이어와의 2025시즌 MLS 정규리그 25라운드 경기에서 2-2 무승부를 거뒀다.손


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap = 52,
    separators = ["\n\n", "\n", " ", ""],
    length_function = len
)

split_docs = text_splitter.split_documents(documents)

print(f"분할 전 Document 수 : {len(documents)}")
print(f"분할 후 Document 수 : {len(split_docs)}")

if split_docs : 
    print("\n ----- 분할 된 첫 청크 -------- ")
    print(split_docs[0].page_content)

분할 전 Document 수 : 1
분할 후 Document 수 : 6

 ----- 분할 된 첫 청크 -------- 
인터 마이애미 리오넬 메시, LAFC 손흥민. /사진=게티이미지코리아[STN뉴스] 반진혁 기자 | 손흥민의 영입 효과를 제대로 누리고 있는 LAFC다.손흥민은 토트넘 홋스퍼 10년 생활을 정리한 후 새로운 도전에 나섰고 LAFC와 동행을 약속했다.LAFC는 지난 7일 손흥민 영입을 발표했다. 계약 기간은 2027년까지이며, 2028, 2029년 6월까지 추가 옵션이 있다.LAFC는 지난 10일 미국 시카고에 위치한 시트긱 스타디움에서 치러진 시카고 파이어와의 2025시즌 MLS 정규리그 25라운드 경기에서 2-2 무승부를 거뒀다.손흥민의 존재감은 번뜩였다. 최전방 공격수로 뛰면서 후반 31분 후방에서 연결된 패스를 받은 후 폭풍 질주를 선보였고 페널티 킥을 얻어냈다. 이는 득점으로 연결됐고 LAFC는 패배를 모면했다.LAFC 손흥민. /사진=게티이미지코리아손흥민의 영입 열기가 뜨겁다.미국 메이저리그 사커(MLS) 전담 기자인 파비안 렌켈은 자신의 개인 채널을 통해 손흥민에 대해 언급했다.렌켈은


In [6]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-nli")

vector_db = FAISS.from_documents(split_docs, embedding_model)
vector_db.save_local("example_math_index")

print("\n🦂 벡터 DB 생성 및 저장 완료 !")
print(f"{embedding_model.model_name} 모델을 사용하여 {len(split_docs)}개의 청크를 생성했습니다.")


🦂 벡터 DB 생성 및 저장 완료 !
jhgan/ko-sbert-nli 모델을 사용하여 6개의 청크를 생성했습니다.


In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os 

In [ ]:
os.environ['OPENAI_API_KEY'] = "sk-"

DB_PATH = "example_math_index"
EMBEDDING_MODEL_NAME = "jhgan/ko-sbert-nli"
print("임베딩 모델과 로컬 벡터 DB를 로딩합니다")
embedding_model = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
vector_db = FAISS.load_local(
    DB_PATH,
    embedding_model,
    allow_dangerous_deserialization=True
)
print("로딩 완료")

retriver = vector_db.as_retriever()
llm = ChatOpenAI(model='gpt-4o', temperature=0.2)

prompt_template = """
### 지시 :
당신은 주어진 문서의 내용을 바탕으로 질문에 답변하는 AI 어시스턴트입니다.
문서에 내용이 없는 경우, '문서에 관련된 내용이 없습니다' 라고 답변하세요, 추측성 답변을 하지 말아주세요.

### 참고 문서 : 
{context}

### 질문 : 
{question}

### 답변 : 

"""


prompt = ChatPromptTemplate.from_template(prompt_template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context" : retriver | format_docs, "question" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


if __name__ == '__main__':
    print("\n RAG 파이프라인을 통해서 질문에 대한 답변을 생성합니다....")

    my_question = "손흥민 선수는 어느 팀 소속이야?"

    response = rag_chain.invoke(my_question)

    print("___최종 답변 _____")
    print(response)

임베딩 모델과 로컬 벡터 DB를 로딩합니다


로딩 완료

 RAG 파이프라인을 통해서 질문에 대한 답변을 생성합니다....
___최종 답변 _____
문서에 따르면 손흥민 선수는 현재 LAFC 소속입니다.


### 위의 예시 처럼 gpt-4o의 경우 2023년 정보까지만 가지고 있습니다. 하지만 RAG를 통해서 최신 정보를 제공해주면 학습하지 않은 정보에 대해서도 정확하게 대답해줄 수 있습니다.